In [ ]:
#!/usr/bin/env python
# coding: utf-8

# # 뉴럴넷으로 패션 아이템 구분하기
# Fashion MNIST 데이터셋과 앞서 배운 인공신경망을 이용하여 패션아이템을 구분해봅니다.
import torch
import torchvision
import torch.nn.functional as F
from torch import nn, optim
from torchvision import transforms, datasets, utils
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import numpy as np
import torch.nn.init as init # 초기화 관련 모듈

import torch.nn as nn
import torch.optim as optim





USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print('Using Device : ',DEVICE)

EPOCHS = 20
BATCH_SIZE = 100


# ## 데이터셋 불러오기

transform = transforms.Compose([
    transforms.ToTensor()
])


trainset = datasets.FashionMNIST(
    root      = './data/',
    train     = True,
    download  = True,
    transform = transform
)
testset = datasets.FashionMNIST(
    root      = './data/',
    train     = False,
    download  = True,
    transform = transform
)

train_loader = torch.utils.data.DataLoader(
    dataset     = trainset,
    batch_size  = BATCH_SIZE,
    shuffle     = False,
)
test_loader = torch.utils.data.DataLoader(
    dataset     = testset,
    batch_size  = BATCH_SIZE,
    shuffle     = False,
)



In [ ]:
dataiter       = iter(train_loader) #배치 1개만 뽑아 데이터가 생긴 형태를 살펴보겠습니다
images, labels = next(dataiter)

# 여러 이미지 모아보기
img   = utils.make_grid(images, padding=0) # util.make_grid() 함수를 이용해 여러 이미지를 하나로 모아 하나의 이미지로 만듭니다
npimg = img.numpy() # img는 파이토치 텐서입니다. numpy()함수로 맷플롯립과 호환이 되는 넘파이 행렬로 바꿔줍니다.
plt.figure(figsize=(10, 10))
plt.imshow(np.transpose(npimg, (1,2,0))) #np.transpose() 함수를 이용해 첫 번째(0번째) 차원을 맨 뒤로 보냅니다
plt.show()
# 여러 개의 패션 아이템이 흑백으로 나열되는 것을 볼 수 있습니다

# classificaiton fashion MNIST

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model        = Net().to(DEVICE)
optimizer    = optim.SGD(model.parameters(), lr=0.01)

def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # 학습 데이터를 DEVICE의 메모리로 보냄
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 모든 오차 더하기
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()

            # 가장 큰 값을 가진 클래스가 모델의 예측입니다.
            # 예측과 정답을 비교하여 일치할 경우 correct에 1을 더합니다.
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)

    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

# Autoencoders

In [ ]:
## Model1

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(28*28, 144),
            nn.ReLU(),
            nn.Linear(144, 64),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16, 3),   # 입력의 특징을 3차원으로 압축합니다
        )


    def forward(self, x):
        encoded = self.encoder(x)
        return encoded

class Decoder(nn.Module):
    def __init__(self) -> None:
        super(Decoder,self).__init__()
        self.decoder = nn.Sequential(
            nn.Linear(3, 16),
            nn.ReLU(),
            nn.Linear(16, 64),
            nn.ReLU(),
            nn.Linear(64, 144),
            nn.ReLU(),
            nn.Linear(144, 28*28),
            nn.Sigmoid(),       # 픽셀당 0과 1 사이로 값을 출력합니다
        )

    def forward(self,z):
        decoded = self.decoder(z)
        return decoded

class Descriminator(nn.Module):
    def __init__(self) -> None:
        super(Descriminator,self).__init__()
        self.des = nn.Sequential(
            nn.Linear(28*28,124),
            nn.ReLU(),
            nn.Linear(124,64),
            nn.ReLU(),
            nn.Linear(64,12),
            nn.ReLU(),
            nn.Linear(12,2)
        )
    def forward(self,x):
        x = self.des(x)
        return x

# Model2

num_epoch = 200
batch_size = 100
learning_rate = 0.0002
img_size = 28 * 28
num_channel = 1
dir_name = "GAN_results"

noise_size = 3
hidden_size1 = 256
hidden_size2 = 512
hidden_size3 = 1024


# Declares discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.linear1 = nn.Linear(img_size, hidden_size3)
        self.linear2 = nn.Linear(hidden_size3, hidden_size2)
        self.linear3 = nn.Linear(hidden_size2, hidden_size1)
        self.linear4 = nn.Linear(hidden_size1, 1)
        self.leaky_relu = nn.LeakyReLU(0.2)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.leaky_relu(self.linear1(x))
        x = self.leaky_relu(self.linear2(x))
        x = self.leaky_relu(self.linear3(x))
        x = self.linear4(x)
        x = self.sigmoid(x)
        return x


# Declares generator & encoder
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()

        self.linear1 = nn.Linear(noise_size, hidden_size1)
        self.linear2 = nn.Linear(hidden_size1, hidden_size2)
        self.linear3 = nn.Linear(hidden_size2, hidden_size3)
        self.linear4 = nn.Linear(hidden_size3, img_size)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.relu(self.linear3(x))
        x = self.linear4(x)
        x = self.tanh(x)
        return x

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()

        self.linear1 = nn.Linear(img_size,hidden_size3)
        self.linear2 = nn.Linear(hidden_size3,hidden_size2)
        self.linear3 = nn.Linear(hidden_size2,hidden_size1)
        self.linear4 = nn.Linear(hidden_size1,noise_size)
        self.relu = nn.ReLU()
        self.tanh(x)
        self.bn = nn.BatchNorm1d(3)

    def forward(self, x):
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.relu(self.linear3(x))
        x = self.linear4(x)
        x = self.bn(x)
        x = self.tanh(x)
        return x



encoder = Encoder().to(DEVICE) # 정의한 모델을 GPU로 납치
decoder = Decoder().to(DEVICE)
def weight_init(m):
    if isinstance(m, nn.Linear):
        init.kaiming_uniform_(m.weight.data)

encoder.apply(weight_init)
decoder.apply(weight_init)

# descriminator = Descriminator().to(DEVICE)
# descriminator.apply(weight_init)

optimizer1 = torch.optim.Adam(encoder.parameters(), lr=0.005)
optimizer2 = torch.optim.Adam(decoder.parameters(), lr =0.005)
# optimizer2 = torch.optim.Adam(descriminator.parameters(), lr=0.001)
criterion = nn.MSELoss()
# criterion = nn.CrossEntropyLoss() # 이렇게 하면 흑백으로 됨.. 참과 거짓..?

# 원본 이미지를 시각화 하기 (첫번째 열)
view_data = trainset.data[:5].view(-1, 28*28)
view_data = view_data.type(torch.FloatTensor)/255.

def train(encoder,decoder, train_loader):
    encoder.train()
    decoder.train()
    total_loss = 0
    for step, (x, label) in enumerate(train_loader):
        x = x.view(-1, 28*28).to(DEVICE)
        y = x.view(-1, 28*28).to(DEVICE)
        label = label.to(DEVICE)

        encoded= encoder(x)
        decoded= decoder(encoded)
        loss = criterion(decoded, y)
        total_loss+=loss
        optimizer1.zero_grad()
        optimizer2.zero_grad()
        loss.backward()
        optimizer1.step()
        optimizer2.step()
    print('loss :',total_loss)



EPOCH = 7

for epoch in range(1, EPOCH+1):
    print("[Epoch {}]".format(epoch))

    train(encoder,decoder, train_loader)

    # 디코더에서 나온 이미지를 시각화 하기 (두번째 열)
    test_x = view_data.to(DEVICE)
    zz = encoder(test_x)
    decoded_data = decoder(zz)

    # 원본과 디코딩 결과 비교해보기
    f, a = plt.subplots(2, 5, figsize=(5, 2))
    for i in range(5):
        img = np.reshape(view_data.data.numpy()[i],(28, 28))
        a[0][i].imshow(img, cmap='gray')
        a[0][i].set_xticks(()); a[0][i].set_yticks(())

    for i in range(5):
        img = np.reshape(decoded_data.to("cpu").data.numpy()[i], (28, 28))
        a[1][i].imshow(img, cmap='gray')
        a[1][i].set_xticks(()); a[1][i].set_yticks(())
    plt.show()



In [ ]:
class Descriminator(nn.Module):
    def __init__(self) -> None:
        super(Descriminator,self).__init__()
        self.des = nn.Sequential(
            nn.Linear(28*28,124),
            nn.ReLU(),
            nn.Linear(124,64),
            nn.ReLU(),
            nn.Linear(64,12),
            nn.ReLU(),
            nn.Linear(12,2)
        )
    def forward(self,x):
        x = self.des(x)
        return x

num_epoch = 200
batch_size = 100
learning_rate = 0.0002
img_size = 28 * 28
num_channel = 1
dir_name = "GAN_results"

noise_size = 3
hidden_size0 = 64
hidden_size1 = 256
hidden_size2 = 512
hidden_size3 = 1024


# Declares discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.linear1 = nn.Linear(img_size, hidden_size3)
        self.linear2 = nn.Linear(hidden_size3, hidden_size2)
        self.linear3 = nn.Linear(hidden_size2, hidden_size1)
        self.linear4 = nn.Linear(hidden_size1, 1)
        self.leaky_relu = nn.LeakyReLU(0.2)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.leaky_relu(self.linear1(x))
        x = self.leaky_relu(self.linear2(x))
        x = self.leaky_relu(self.linear3(x))
        x = self.linear4(x)
        x = self.sigmoid(x)
        return x


# Declares generator & encoder
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()

        self.linear0 = nn.Linear(noise_size, hidden_size0)
        self.linear1 = nn.Linear(hidden_size0, hidden_size1)
        self.linear2 = nn.Linear(hidden_size1, hidden_size2)
        self.linear3 = nn.Linear(hidden_size2, hidden_size3)
        self.linear4 = nn.Linear(hidden_size3, img_size)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.relu(self.linear0(x))
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.relu(self.linear3(x))
        x = self.linear4(x)
        x = self.tanh(x)
        return x

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()

        self.linear1 = nn.Linear(img_size,hidden_size3)
        self.linear2 = nn.Linear(hidden_size3,hidden_size2)
        self.linear3 = nn.Linear(hidden_size2,hidden_size1)
        self.linear4 = nn.Linear(hidden_size1,hidden_size0)
        self.linear5 = nn.Linear(hidden_size0,noise_size)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.bn = nn.BatchNorm1d(3)

    def forward(self, x):
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.relu(self.linear3(x))
        x = self.relu(self.linear4(x))
        x = self.linear5(x)
        x = self.bn(x)
        x = self.tanh(x)
        return x


discriminator = Discriminator().to(DEVICE)
encoder = Encoder().to(DEVICE) # 정의한 모델을 GPU로 납치
decoder = Decoder().to(DEVICE)

def weight_init(m):
    if isinstance(m, nn.Linear):
        init.kaiming_uniform_(m.weight.data)

encoder.apply(weight_init)
decoder.apply(weight_init)
discriminator.apply(weight_init)

# descriminator = Descriminator().to(DEVICE)
# descriminator.apply(weight_init)

optimizer1 = torch.optim.Adam(encoder.parameters(), lr=0.005)
optimizer2 = torch.optim.Adam(decoder.parameters(), lr =0.005)
# optimizer2 = torch.optim.Adam(descriminator.parameters(), lr=0.001)
criterion = nn.MSELoss()
# criterion = nn.CrossEntropyLoss() # 이렇게 하면 흑백으로 됨.. 참과 거짓..?

# 원본 이미지를 시각화 하기 (첫번째 열)
view_data = trainset.data[:5].view(-1, 28*28)
view_data = view_data.type(torch.FloatTensor)/255.

def train(encoder,decoder, train_loader):
    encoder.train()
    decoder.train()
    total_loss = 0
    for step, (x, label) in enumerate(train_loader):
        x = x.view(-1, 28*28).to(DEVICE)
        y = x.view(-1, 28*28).to(DEVICE)
        label = label.to(DEVICE)

        encoded= encoder(x)
        decoded= decoder(encoded)
        loss = criterion(decoded, y)
        total_loss+=loss
        optimizer1.zero_grad()
        optimizer2.zero_grad()
        loss.backward()
        optimizer1.step()
        optimizer2.step()
    print('loss :',total_loss)



EPOCH = 3

for epoch in range(1, EPOCH+1):
    print("[Epoch {}]".format(epoch))

    train(encoder,decoder, train_loader)

    # 디코더에서 나온 이미지를 시각화 하기 (두번째 열)
    test_x = view_data.to(DEVICE)
    zz = encoder(test_x)
    decoded_data = decoder(zz)

    # 원본과 디코딩 결과 비교해보기
    f, a = plt.subplots(2, 5, figsize=(5, 2))
    for i in range(5):
        img = np.reshape(view_data.data.numpy()[i],(28, 28))
        a[0][i].imshow(img, cmap='gray')
        a[0][i].set_xticks(()); a[0][i].set_yticks(())

    for i in range(5):
        img = np.reshape(decoded_data.to("cpu").data.numpy()[i], (28, 28))
        a[1][i].imshow(img, cmap='gray')
        a[1][i].set_xticks(()); a[1][i].set_yticks(())
    plt.show()



# 잠재 변수 보기


In [ ]:
# 잠재변수를 3D 플롯으로 시각화
view_data = trainset.data[:200].view(-1, 28*28)
view_data = view_data.type(torch.FloatTensor)/255.
test_x = view_data.to(DEVICE)
encoded_data = encoder(test_x)
encoded_data = encoded_data.to("cpu")

CLASSES = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boot'
}

# 데이터 생성

fig = plt.figure(figsize=(10,8))
ax = Axes3D(fig)

X = encoded_data.data[:, 0].numpy()
Y = encoded_data.data[:, 1].numpy()
Z = encoded_data.data[:, 2].numpy()

labels = trainset.targets[:200].numpy()
# 클래스 이름과 색상 매핑
cm = plt.cm.rainbow

# 3D 플롯 생성
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# 각 데이터 포인트에 클래스 이름과 색상을 지정
for x, y, z, s in zip(X, Y, Z, labels):
    name = CLASSES[s]
    color = cm(int(255*s/9))
    ax.text(x, y, z, name, backgroundcolor=color)

# 축 범위 설정
ax.set_xlim(X.min(), X.max())
ax.set_ylim(Y.min(), Y.max())
ax.set_zlim(Z.min(), Z.max())

# 플롯 표시
plt.show()

In [ ]:
# VAE로 만들어진 Z를 sampling해서 다시 데이터 셋으로 만들기.

sampling_z = torch.randn(0)
for x,y in train_loader:
    # print(x.shape)
    # print(y.shape)
    x = x.view(-1,784).to(DEVICE)
    z = encoder(x)
    z = z.detach().cpu().type(torch.float32)
    sampling_z = torch.concat((sampling_z,z),dim=0)
print(sampling_z.shape)

# noise
noise = torch.randn_like(sampling_z)*0.2
noise_sampling_z = noise +sampling_z

z_loader = torch.utils.data.DataLoader(
    dataset     = sampling_z, # not noisy
    batch_size  = BATCH_SIZE,
    shuffle     = False,
)

noisy_z_loader = torch.utils.data.DataLoader(
    dataset     = noise_sampling_z, #noisy
    batch_size  = BATCH_SIZE,
    shuffle     = False,
)


# GAN

위 샘플링된 z를 통해서 GAN을 할것이다. 원래 기본코드와 다르게 간닷.

In [ ]:
for step,((x,label),z,noisy_z) in enumerate(zip(train_loader,z_loader,noisy_z_loader)):
  print(x.shape)
  print(label)
  print(z.shape)
  print(label.get_device())
  print(z.get_device()) # 0이면 gpu -1이면 cpu
  print(noisy_z.get_device()) # 0이면 gpu -1이면 cpu
  break

In [ ]:
# 잘 만들어지나 실험.

for step,((x,label),z,noisy_z) in enumerate(zip(train_loader,z_loader,noisy_z_loader)):
  print(x.shape)
  print(label)
  print(z.shape)
  fig = plt.figure(figsize = (10,10))
  number_item = 19
  plt.subplot(1,3,1)
  plt.title('real')
  plt.imshow(x[number_item].view(28,28))

  # z to decode
  z.to('cpu')
  decoder.to('cpu')
  yyy = decoder(z[number_item])
  print(yyy.shape)

  plt.subplot(1,3,2)
  plt.title('decoded')
  plt.imshow(yyy.view(28,28).detach().cpu())


  # noisy z to decode
  noisy_z.to('cpu')
  yyy2 = decoder(noisy_z[number_item])
  print(yyy2.shape)

  plt.subplot(1,3,3)
  plt.title('decoded noisy z')
  plt.imshow(yyy2.view(28,28).detach().cpu())
  break

In [ ]:
discriminator = Discriminator().to(DEVICE)
decoder = decoder.to(DEVICE)

discriminator.apply(weight_init)


criterion = nn.BCELoss()

d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(decoder.parameters(), lr =0.0002) # generator

# 원본 이미지를 시각화 하기 (첫번째 열)
view_data = trainset.data[:5].view(-1, 28*28)
view_data = view_data.type(torch.FloatTensor)/255.


EPOCH = 200
device = DEVICE
batch_size = 100
for epoch in range(1, EPOCH+1):
    for step,((x,label),z,noisy_z) in enumerate(zip(train_loader,z_loader,noisy_z_loader)):
        real_label = torch.full((batch_size, 1), 1, dtype=torch.float32).to(device)
        fake_label = torch.full((batch_size, 1), 0, dtype=torch.float32).to(device)

        real_images = images.reshape(batch_size, -1).to(device)

        # +---------------------+
        # |   train Generator   |
        # +---------------------+

        # Initialize grad
        g_optimizer.zero_grad()
        d_optimizer.zero_grad()

        z = z.detach().to(device)
        # z = torch.randn(batch_size, noise_size).to(device)
        fake_images = decoder(z)

        # Compare result of discriminator with fake images & real labels
        # If generator deceives discriminator, g_loss will decrease
        g_loss = criterion(discriminator(fake_images), real_label)

        # Train generator with backpropagation
        g_loss.backward()
        g_optimizer.step()

        # +---------------------+
        # | train Discriminator |
        # +---------------------+

        # Initialize grad
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()

        # make fake images with generator & noise vector 'z'
        noisy_z = noisy_z.detach().to(device)
        # noisy_z = torch.randn(batch_size, noise_size).to(device)
        fake_images = decoder(noisy_z)

        # Calculate fake & real loss with generated images above & real images
        fake_loss = criterion(discriminator(fake_images), fake_label)
        real_loss = criterion(discriminator(real_images), real_label)
        d_loss = (fake_loss + real_loss) / 2

        # Train discriminator with backpropagation
        # In this part, we don't train generator
        d_loss.backward()
        d_optimizer.step()

        d_performance = discriminator(real_images).mean()
        g_performance = discriminator(fake_images).mean()

        if (i + 1) % 150 == 0:
            print("Epoch [ {}/{} ]  Step [ {}/{} ]  d_loss : {:.5f}  g_loss : {:.5f}"
                  .format(epoch, num_epoch, i+1, len(train_loader), d_loss.item(), g_loss.item()))

    # print discriminator & generator's performance
    print(" Epock {}'s discriminator performance : {:.2f}  generator performance : {:.2f}"
          .format(epoch, d_performance, g_performance))

    # images test
    test_x = view_data.to(DEVICE)
    zz = encoder(test_x)
    decoded_data = decoder(zz)

    # 원본과 디코딩 결과 비교해보기
    f, a = plt.subplots(2, 5, figsize=(5, 2))
    for i in range(5):
        img = np.reshape(view_data.data.numpy()[i],(28, 28))
        a[0][i].imshow(img, cmap='gray')
        a[0][i].set_xticks(()); a[0][i].set_yticks(())

    for i in range(5):
        img = np.reshape(decoded_data.to("cpu").data.numpy()[i], (28, 28))
        a[1][i].imshow(img, cmap='gray')
        a[1][i].set_xticks(()); a[1][i].set_yticks(())
    plt.show()